Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE       MCSE      ESS
alpha 154.5946747 0.277604625 0.0043893145 0.0037100823 1000
 beta   0.9043439 0.042560753 0.0006729446 0.0006217203 1000
sigma   5.1069639 0.191383972 0.0030260463 0.0025257923 1000

Quantiles:
          2.5%        25.0%       50.0%      75.0%        97.5%   
alpha 154.0509750 154.41200000 154.595000 154.7820000 155.14405000
 beta   0.8214199   0.87601275   0.905165   0.9332620   0.98798085
sigma   4.7591995   4.97187250   5.100050   5.2323525   5.51246025



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.783 0.939854 5.14611; 154.331 0.92641 5.24755; … ; 154.566 0.9687 5.08858; 154.555 0.892899 5.07218]

Union{Missing, Float64}[155.036 0.952812 5.2956; 155.056 0.86069 5.2173; … ; 154.404 0.862588 4.89356; 154.525 0.889904 5.4328]

Union{Missing, Float64}[154.868 0.932931 4.80337; 154.268 0.885611 4.84433; … ; 154.484 0.866865 5.04388; 154.587 0.940507 4.97042]

Union{Missing, Float64}[154.485 0.870405 4.9661; 154.693 0.910942 4.8489; … ; 155.008 0.923152 5.27618; 154.257 0.87972 4.88721]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE       MCSE      ESS
alpha 154.5946747 0.277604625 0.0043893145 0.0037100823 1000
 beta   0.9043439 0.042560753 0.0006729446 0.0006217203 1000
sigma   5.1069639 0.191383972 0.0030260463 0.0025257923 1000

Quantiles:
          2.5%        25.0%       50.0%      75.0%        97.5%   
alpha 154.0509750 154.41200000 154.595000 154.7820000 155.14405000
 beta   0.8214199   0.87601275   0.905165   0.9332620   0.98798085
sigma   4.7591995   4.97187250   5.100050   5.2323525   5.51246025



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*